In [41]:
import numpy as np
import pandas as pd
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import keras_tuner as kt

In [2]:
names = ['subject-id', 'activity-code', 'timestamp_phone_accel', 'x_phone_accel', 'y_phone_accel', 'z_phone_accel']

path = r'wisdm-dataset/raw/phone/accel' # use your path
all_files = glob.glob(path + "/*.txt")

li = []

for filename in all_files:
    df = pd.read_csv(filename, names=names)
    li.append(df)

frame_phone_accel = pd.concat(li, axis=0, ignore_index=True).drop(['subject-id', 'timestamp_phone_accel'], axis=1)

frame_phone_accel['z_phone_accel'] = frame_phone_accel['z_phone_accel'].str.rstrip(';').astype('float64')

In [3]:
names = ['subject-id', 'activity-code', 'timestamp_phone_gyro', 'x_phone_gyro', 'y_phone_gyro', 'z_phone_gyro']

path = r'wisdm-dataset/raw/phone/gyro' # use your path
all_files = glob.glob(path + "/*.txt")

li = []

for filename in all_files:
    df = pd.read_csv(filename, names=names)
    li.append(df)

frame_phone_gyro = pd.concat(li, axis=0, ignore_index=True).drop(['subject-id', 'timestamp_phone_gyro'], axis=1)
frame_phone_gyro['z_phone_gyro'] = frame_phone_gyro['z_phone_gyro'].str.rstrip(';').astype('float64')

In [4]:
names = ['subject-id', 'activity-code', 'timestamp_watch_accel', 'x_watch_accel', 'y_watch_accel', 'z_watch_accel']

path = r'wisdm-dataset/raw/watch/accel' # use your path
all_files = glob.glob(path + "/*.txt")

li = []

for filename in all_files:
    df = pd.read_csv(filename, names=names)
    li.append(df)

frame_watch_accel = pd.concat(li, axis=0, ignore_index=True).drop(['subject-id', 'timestamp_watch_accel'], axis=1)
frame_watch_accel['z_watch_accel'] = frame_watch_accel['z_watch_accel'].str.rstrip(';').astype('float64')

In [5]:
names = ['subject-id', 'activity-code', 'timestamp_watch_gyro', 'x_watch_gyro', 'y_watch_gyro', 'z_watch_gyro']

path = r'wisdm-dataset/raw/watch/gyro' # use your path
all_files = glob.glob(path + "/*.txt")

li = []

for filename in all_files:
    df = pd.read_csv(filename, names=names)
    li.append(df)

frame_watch_gyro = pd.concat(li, axis=0, ignore_index=True).drop(['subject-id', 'timestamp_watch_gyro'], axis=1)
frame_watch_gyro['z_watch_gyro'] = frame_watch_gyro['z_watch_gyro'].str.rstrip(';').astype('float64')

In [83]:
X_train_full_phone_accel, X_test_phone_accel, y_train_full_phone_accel, y_test_phone_accel = train_test_split(frame_phone_accel[['x_phone_accel', 'y_phone_accel', 'z_phone_accel']], frame_phone_accel[['activity-code']], test_size=0.2, random_state=150, stratify=frame_phone_accel['activity-code'])
X_train_phone_accel, X_valid_phone_accel, y_train_phone_accel, y_valid_phone_accel = train_test_split(X_train_full_phone_accel, y_train_full_phone_accel, test_size=X_test_phone_accel.shape[0], random_state=150)

In [84]:
scaler = MinMaxScaler().set_output(transform='pandas')
scaled_X_train_phone_accel = scaler.fit_transform(X_train_phone_accel) 
scaled_X_valid_phone_accel = scaler.transform(X_valid_phone_accel)
scaled_X_test_phone_accel = scaler.transform(X_test_phone_accel)

In [85]:
encoder = OneHotEncoder(sparse=False).set_output(transform='pandas')
y_train_phone_accel_cat = encoder.fit_transform(y_train_phone_accel)
y_valid_phone_accel_cat = encoder.transform(y_valid_phone_accel)
y_test_phone_accel_cat = encoder.transform(y_test_phone_accel)

c:\Users\Shaun\miniconda3\envs\WISDM\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [116]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=1, max_value=5)
    n_neurons = hp.Int("n_neurons", min_value=10, max_value=100)
    learning_rate = hp.Float("learning_rate", min_value=1e-6, max_value=10, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "rmsprop", "adam", "adamax", "nadam", "adamw"])

    if optimizer == "sgd":
        momentum = hp.Float('momentum', min_value=0, max_value=1)
        nesterov = hp.Boolean('nesterov')
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)

    elif optimizer == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

    elif optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    elif optimizer == "adamax":
        optimizer = tf.keras.optimizers.Adamax(learning_rate=learning_rate)

    elif optimizer == "nadam":
        optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

    else:
        optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)


    inp = tf.keras.Input(shape=(3,))
    hidden_layer_dict = {}
    hidden_layer_dict['hidden1'] = tf.keras.layers.Dense(n_neurons, activation='relu')(inp)
    for i in range(2, n_hidden + 1):
        hidden_layer_dict['hidden'+str(i)] = tf.keras.layers.Dense(n_neurons, activation='relu')(hidden_layer_dict['hidden'+str(i-1)])
    

    out = tf.keras.layers.Dense(18, activation='softmax', name='Activity')(hidden_layer_dict['hidden'+str(n_hidden)])

    model = tf.keras.Model(inp, out)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [117]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)
    
    def fit(self, hp, model, X, y, **kwargs):
        batch_size = hp.Int('batch_size', min_value=2, max_value=32)
        return model.fit(X, y, batch_size=batch_size, **kwargs)


In [118]:
bayesian_opt_tuner = kt.BayesianOptimization(MyHyperModel(), objective='val_loss', seed=150, max_trials=100, overwrite=False, directory='wisdm', project_name='my_bayesian_search')

Reloading Tuner from wisdm\my_bayesian_search\tuner0.json


In [119]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

In [120]:
bayesian_opt_tuner.search(scaled_X_train_phone_accel.head(1000), y_train_phone_accel_cat.head(1000), validation_data=(scaled_X_valid_phone_accel.head(1000), y_valid_phone_accel_cat.head(1000)), epochs=500, callbacks=[early_stopping_cb])

Trial 100 Complete [00h 00m 04s]
val_loss: 2.890199899673462

Best val_loss So Far: 2.6207170486450195
Total elapsed time: 00h 17m 07s
